<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-01 14:28:29
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -13.89 K (-0.1%)
Current PnL: -13.71 L (-9.55%)
CY Booked + Current PnL: -6.97 L (-4.85%)
-------------------
Total profit:  4.45 L
Total loss:  -18.16 L
-------------------
Total Booked + Current PnL: 20.01 L (17.17%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.13 L (52.96%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.00,1.69,M-LC,46.88,176406.0,35566.0,8926.0,1.01,25.25,5.06,31.59,89.0,3.98,1.30,53.23,XY24,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,91.67,133812.0,-8667.0,9287.0,1.08,-6.08,6.94,0.44,243.0,-0.93,0.98,57.93,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,27.08,102659.0,24297.0,9732.0,-0.30,31.01,9.48,43.43,77.0,2.50,0.75,37.97,XY24,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,86.46,111024.0,6712.0,10825.0,0.15,6.43,9.75,16.81,27.0,0.62,0.82,23.51,X5K,ATH,METALS
71,SBIN,760.30,863.00,-9.61,M-LC,44.27,217777.0,16298.0,10911.0,0.18,8.09,5.01,13.51,76.0,1.49,1.60,21.70,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,141994.0,152.0,56414.0,2.46,0.11,39.73,39.88,259.0,0.00,1.04,43.90,XY24,NTT,POWER
60,NATIONALUM,189.63,247.44,-50.15,H-MC,65.62,100827.0,323.0,30319.0,-1.28,0.32,30.07,30.49,97.0,0.01,0.74,32.83,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,1646.70,2337.55,-9.41,H-SC,85.42,169890.0,5220.0,63862.0,3.87,3.17,37.59,41.95,121.0,0.08,1.25,10.87,X40N,ATH,AC
79,SYMPHONY,1306.42,1306.00,-31.65,M-SC,6.25,145606.0,-25535.0,25481.0,3.38,-14.92,17.50,-0.03,189.0,-1.00,1.07,4.65,OX40N,NTT,DURABLES
5,APOLLOHOSP,6904.43,8285.00,-17.74,H-LC,78.12,112140.0,8574.0,12134.0,3.23,8.28,10.82,20.00,30.0,0.71,0.82,23.52,X40N,BTT,HEALTHCARE
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,141994.0,152.0,56414.0,2.46,0.11,39.73,39.88,259.0,0.00,1.04,43.90,XY24,NTT,POWER
12,BANKINDIA,116.91,190.00,-28.83,M-MC,39.58,186621.0,6813.0,105590.0,2.34,3.79,56.58,62.52,171.0,0.06,1.37,39.17,XR,NTT,BANKS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,15978.0,-33615.0,2539.0,-3.93,-67.78,15.89,-62.66,NaN,-13.24,0.12,7.15,XY24,ATH,MISC
29,DIXON,14367.00,18940.15,-36.54,H-MC,34.38,116872.0,1936.0,34653.0,-2.50,1.68,29.65,31.83,99.0,0.06,0.86,33.95,X40N,ATH,IT
1,ABB,5551.76,7934.00,-35.10,H-LC,29.17,231504.0,14985.0,77924.0,-2.38,6.92,33.66,42.91,28.0,0.19,1.70,21.01,AR,NTT,ELECTRICAL
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,97.92,99438.0,-1525.0,96057.0,-2.34,-1.51,96.60,93.63,64.0,-0.02,0.73,48.11,SR,ATH,CHEMICALS
61,NESTLEIND,2268.60,2755.00,-11.07,X-LC,36.46,282122.0,16696.0,40202.0,-2.19,6.29,14.25,21.44,9.0,0.42,2.07,12.88,XY25,NTT,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,91.67,133812.0,-8667.0,9287.0,1.08,-6.08,6.94,0.44,243.0,-0.93,0.98,57.93,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-215.53,H-SC,96.88,83362.0,-11954.0,11996.0,1.51,-12.54,14.39,0.04,154.0,-1.00,0.61,64.42,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-31.65,M-SC,6.25,145606.0,-25535.0,25481.0,3.38,-14.92,17.50,-0.03,189.0,-1.00,1.07,4.65,OX40N,NTT,DURABLES
81,TATAELXSI,7332.28,7332.0,-17.48,X-MC,20.31,86695.0,-15957.0,15952.0,-1.86,-15.54,18.40,-0.00,58.0,-1.00,0.64,31.79,OX40N,NTT,IT
46,INDIGOPNTS,1407.73,1408.0,123.21,H-SC,87.50,146506.0,-28053.0,28085.0,1.66,-16.07,19.17,0.02,127.0,-1.00,1.08,27.04,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42779.0,1524.0,9283.0,-0.40,3.69,21.70,26.19,1.0,0.16,0.31,0.00,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,33.33,266588.0,-29346.0,123244.0,-1.28,-9.92,46.23,31.73,1.0,-0.24,1.96,6.71,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,58.33,287564.0,25963.0,119655.0,0.29,9.92,41.61,55.67,2.0,0.22,2.11,16.64,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-18.17,X-LC,21.88,256491.0,-13856.0,48477.0,-0.20,-5.13,18.90,12.81,6.0,-0.29,1.88,7.27,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.07,X-LC,36.46,282122.0,16696.0,40202.0,-2.19,6.29,14.25,21.44,9.0,0.42,2.07,12.88,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,1.69,M-LC,46.88,176406.0,35566.0,8926.0,1.01,25.25,5.06,31.59,89.0,3.98,1.30,53.23,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,27.08,102659.0,24297.0,9732.0,-0.30,31.01,9.48,43.43,77.0,2.50,0.75,37.97,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,72.92,73075.0,38626.0,1323.0,-1.78,112.13,1.81,115.97,NaN,29.20,0.54,11.74,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.10,M-MC,50.00,229594.0,4632.0,163012.0,1.02,2.06,71.00,74.53,185.0,0.03,1.69,33.30,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,29.50,H-SC,76.04,132615.0,7318.0,51892.0,2.25,5.84,39.13,47.25,139.0,0.14,0.97,65.91,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.65,H-SC,54.17,118600.0,16596.0,28428.0,-1.27,16.27,23.97,44.14,124.0,0.58,0.87,35.15,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,27.08,102659.0,24297.0,9732.0,-0.30,31.01,9.48,43.43,77.0,2.50,0.75,37.97,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,1.69,M-LC,46.88,176406.0,35566.0,8926.0,1.01,25.25,5.06,31.59,89.0,3.98,1.30,53.23,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,72.92,73075.0,38626.0,1323.0,-1.78,112.13,1.81,115.97,NaN,29.20,0.54,11.74,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.68,M-SC,89.58,202316.0,30695.0,131364.0,-1.79,17.89,64.93,94.43,198.0,0.23,1.49,49.32,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.16,29.50,H-SC,76.04,132615.0,7318.0,51892.0,2.25,5.84,39.13,47.25,139.0,0.14,0.97,65.91,MH,ATH,POWER
95,WIPRO,243.46,420.00,-4.22,M-LC,71.88,163897.0,12952.0,96503.0,-0.61,8.58,58.88,72.51,69.0,0.13,1.20,12.90,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,92.71,137275.0,13939.0,119827.0,-0.30,11.30,87.29,108.45,122.0,0.12,1.01,36.18,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,98.96,70848.0,-42701.0,82750.0,-1.47,-37.61,116.80,35.27,261.0,-0.52,0.52,73.40,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.25,H-SC,63.54,92730.0,-5250.0,27986.0,0.34,-5.36,30.18,23.21,137.0,-0.19,0.68,34.49,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,97.92,99438.0,-1525.0,96057.0,-2.34,-1.51,96.60,93.63,64.0,-0.02,0.73,48.11,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,27.08,102659.0,24297.0,9732.0,-0.30,31.01,9.48,43.43,77.0,2.50,0.75,37.97,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-350.78,H-SC,5.21,106600.0,-26686.0,111280.0,0.06,-20.02,104.39,63.47,148.0,-0.24,0.78,78.69,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,141994.0,152.0,56414.0,2.46,0.11,39.73,39.88,259.0,0.00,1.04,43.90,XY24,NTT,POWER
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,98.96,70848.0,-42701.0,82750.0,-1.47,-37.61,116.80,35.27,261.0,-0.52,0.52,73.40,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,97.92,99438.0,-1525.0,96057.0,-2.34,-1.51,96.60,93.63,64.0,-0.02,0.73,48.11,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-215.53,H-SC,96.88,83362.0,-11954.0,11996.0,1.51,-12.54,14.39,0.04,154.0,-1.00,0.61,64.42,OX40N,NTT,MISC
51,JIOFIN,310.58,387.00,8.02,H-LC,95.83,231420.0,14014.0,39480.0,1.18,6.45,17.06,24.61,95.0,0.35,1.70,64.54,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.80
1,25,41.86
2,50,70.42


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.52
LC    34.05
MC    21.42
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.53
X40      12.23
XY25     12.01
X40N     11.77
XR       10.15
AR        8.26
OX40N     8.20
MH        1.71
X5K       1.48
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.66
M-SC    16.02
H-LC    14.15
X-LC    12.17
H-MC     9.46
X-MC     7.50
M-LC     6.67
X-SC     4.36
M-MC     4.09
L-SC     2.48
L-LC     1.06
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.06,-2.03,36.14
IT,12.04,-10.76,65.18
BANKS,8.02,-4.43,47.04
FINANCE,7.94,-11.41,51.73
MISC,6.62,-15.12,68.43
PAINTS,5.68,-12.99,35.26
ELECTRICAL,5.48,1.12,45.05
HEALTHCARE,4.51,-4.88,35.52
AUTO,4.30,-15.38,66.11


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2709169.0,27
XR,1075556.0,13
AR,1019639.0,10
X40,655423.0,9
X40N,560103.0,11
XY25,483927.0,8
OX40N,312871.0,11
SR,177400.0,2
MH,82211.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1968194.0,20
M-SC,1762087.0,21
H-LC,612421.0,12
H-MC,546474.0,8
X-LC,517366.0,7
X-MC,432588.0,6
M-MC,413636.0,3
X-SC,324178.0,5
M-LC,264978.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      859124.0      6
M-SC       XY24      813894.0      7
H-SC       AR        518628.0      3
M-SC       XR        335046.0      4
M-MC       XY24      308046.0      2
H-MC       XY24      299148.0      3
X-LC       X40       242899.0      2
H-SC       X40N      229654.0      4
           XR        218387.0      3
H-LC       X40N      200658.0      4
M-SC       AR        181879.0      2
           OX40N     166422.0      5
H-LC       X40       158064.0      3
M-LC       XY24      157564.0      3
X-MC       X40       152969.0      2
L-SC       XR        144312.0      2
X-SC       XR        123522.0      1
X-LC       XY25      121580.0      3
           AR        116269.0      1
M-SC       XY25      111384.0      1
X-MC       XY25      106081.0      1
M-MC       XR        105590.0      1
X-MC       XY24       99066.0      1
M-LC       XR         96503.0      1
X-SC       SR         96057.0      1
H-SC       OX40N      90509.0      3
H-MC       XY25       88776.0      1
M-SC       SR         81343.0      1
H-LC       AR         77924.0      1
M-SC       X40        72119.0      1
H-LC       X200       69489.0      1
           X5K        66806.0      2
X-SC       XY24       64611.0      1
H-MC       AR         64206.0      1
L-SC       AR         59410.0      1
X-MC       X40N       58520.0      1
L-SC       XY24       56414.0      1
L-MC       XR         52196.0      1
H-SC       MH         51892.0      1
L-LC       XY25       45195.0      1
X-SC       OX40N      39988.0      2
H-LC       XY24       39480.0      1
X-LC       X40N       36618.0      1
H-MC       X40N       34653.0      1
           MH         30319.0      1
           X40        29372.0      1
X-MC       OX40N      15952.0      1
M-LC       XY25       10911.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
